# Manipulating/Composing structures 

## Bulding graphene structures

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from scipy.stats import multivariate_normal

from ase import Atoms
from ase.io import write
from ase.build import make_supercell
from ase.lattice.hexagonal import Graphene

from pymatgen.io.ase import AseAtomsAdaptor

from jupyter_jsmol import JsmolView
from jupyter_jsmol.pymatgen import quick_view

Lattice paramter and the height of the cell:

In [ ]:
# Paramteres

a = 2.46
c = 5.

Building from primitive cell

method 1:

In [ ]:
atoms = Graphene('C', latticeconstant={'a':a,'c':c})

In [ ]:
JsmolView.from_ase(make_supercell(atoms, np.diag([5,5,1])))
# quick_view(AseAtomsAdaptor().get_structure(atoms), supercell=(5,5,1))

method 2:

In [ ]:
atoms = Atoms(
    symbols='C2', 
    pbc=[True, True, False],
    cell=[(np.sqrt(3) / 2 * a,  a / 2, 0),
          (np.sqrt(3) / 2 * a, -a / 2, 0),
          (               0.0,    0.0, c)], 
    positions=[
        ( np.sqrt(3) / 6 * a, a / 2, 0.0),
        (-np.sqrt(3) / 6 * a, a / 2, 0.0)
    ]

)

super_atoms = make_supercell(atoms, np.diag([5,5,1]))

In [ ]:
coords = super_atoms.get_positions()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(coords[:,0],coords[:,1])
ax.axis('equal');

In [ ]:
from ase.visualize import view

view(super_atoms, viewer='x3d')

## Introducing corrugation

The depth of the corrugation can approximated with a gaussian function:

In [ ]:
# centered multivariate gaussian function
center = super_atoms.get_center_of_mass()
mean = center[0:2]
cov = np.diag([4.,4.])

rv = multivariate_normal(mean, cov)

In [ ]:
# visualisation of the gaussian function

x, y = np.mgrid[min(coords[:,0]):max(coords[:,0]):.01, min(coords[:,1]):max(coords[:,1]):.01]
pos = np.dstack((x, y))

fig = plt.figure()
ax = fig.add_subplot(111)
ax.contourf(x, y, rv.pdf(pos))
ax.axis('equal')

In [ ]:
z_comp = rv.pdf(coords[:, 0:2])
z_comp /= max(z_comp)

In [ ]:
A = 2.5 # amplitude
super_atoms.arrays["positions"][:,2] = A * z_comp

In [ ]:
from ase.visualize import view

view(super_atoms, viewer='x3d')